In [ ]:
import platform_alerts_functions as af
import warnings
warnings.filterwarnings("ignore")

username = ''
token = ''

array = 'RS'

RS_recipients = ['knuth@marine.rutgers.edu']
CE_recipients = ['knuth@marine.rutgers.edu']
GA_recipients = ['knuth@marine.rutgers.edu']
CP_recipients = ['knuth@marine.rutgers.edu']


stream_most_recent = af.get_most_recent_eng(array)
# alert_deployment_data = af.alert_request_eng_deployments(array,username,token)
not_operational = af.request_annotations(array, username, token)
request_urls,request_inputs = af.alert_build_eng_requests(array)
eng_streams_data = af.send_eng_data_requests(array,request_urls,username,token,
                                             RS_recipients,
                                             CE_recipients,
                                             GA_recipients,
                                             CP_recipients)
stream_final = af.alert_create_eng_outputs(array,eng_streams_data,request_inputs)
stream_final_out = af.alert_create_missing_output(array,stream_final)
no_data_not_annotated,annotated_and_not_operational = af.compare_operational(not_operational, stream_final_out)
stream_difference_new,stream_difference_resumed = af.stream_compare_output(array, stream_final_out, stream_most_recent, request_inputs)
af.alert_send(array,
          no_data_not_annotated,
          annotated_and_not_operational,
          stream_difference_new,
          stream_difference_resumed,
          RS_recipients,
          CE_recipients,
          GA_recipients,
          CP_recipients)


In [ ]:
import netCDF4 as nc
import datetime
import requests
import json
anno_base_url = 'https://ooinet.oceanobservatories.org/api/m2m/12580/anno/find?'
refdes = 'RS01SBPS-SF01A-2A-CTDPFA102'
base_url = 'https://ooinet.oceanobservatories.org/api/m2m/12576/sensor/inv/'
beginDT = int(nc.date2num(datetime.datetime.strptime("2012-01-01T01:00:01Z",'%Y-%m-%dT%H:%M:%SZ'),'seconds since 1970-01-01')*1000)
endDT = int(nc.date2num(datetime.datetime.utcnow(),'seconds since 1970-01-01')*1000)

data_request_url = anno_base_url + 'beginDT='+str(beginDT)+'&endDT='+str(endDT)+'&refdes='+refdes
r = requests.get(data_request_url, auth=(username, token))
data = r.json()

print(json.dumps(data, indent=2))